In [1]:
 !rm -r /content/gansynth &>/dev/null
 !mkdir /content/gansynth
 !mkdir /content/gansynth/midi
 !mkdir /content/gansynth/samples
 # Load default MIDI (Bach Prelude)
 #’curl’ command enables fetching a given URL 
#  !curl -o /content/gansynth/midi/bach.mid http://www.jsbach.net/midi/cs1-1pre.mid 
!curl -o /content/gansynth/midi/bach.mid http://www.jsbach.net/midi/1080-c01.mid

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6952  100  6952    0     0  16205      0 --:--:-- --:--:-- --:--:-- 16167


In [2]:
 SONG = '/content/gansynth/midi/bach.mid'
 !curl -o /content/gansynth/midi/riff-default.mid http://storage.googleapis.com/magentadata/papers/gansynth/midi/arp.mid
 RIFF = '/content/gansynth/midi/riff-default.mid'
 !pip install -q -U magenta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1083  100  1083    0     0   3332      0 --:--:-- --:--:-- --:--:--  3332
     |████████████████████████████████| 1.4 MB 15.5 MB/s 
     |████████████████████████████████| 5.6 MB 15.8 MB/s 
     |████████████████████████████████| 254 kB 43.1 MB/s 
     |████████████████████████████████| 210 kB 43.4 MB/s 
     |████████████████████████████████| 3.6 MB 42.7 MB/s 
     |████████████████████████████████| 69 kB 5.3 MB/s 
     |████████████████████████████████| 204 kB 21.9 MB/s 
     |████████████████████████████████| 90 kB 4.4 MB/s 
     |████████████████████████████████| 1.4 MB 36.5 MB/s 
     |████████████████████████████████| 2.3 MB 33.9 MB/s 
     |████████████████████████████████| 1.6 MB 43.2 MB/s 
     |████████████████████████████████| 352 kB 45.0 MB/s 
     |████████████████████████████████| 20.2 MB 1.4 MB/s 
     |███████

In [3]:
 import os
 from google.colab import files 
 import librosa
 from magenta.models.nsynth.utils import load_audio
 from magenta.models.gansynth.lib import flags as lib_flags
 from magenta.models.gansynth.lib import generate_util as gu
 from magenta.models.gansynth.lib import model as lib_model
 from magenta.models.gansynth.lib import util
 import matplotlib.pyplot as plt
 import note_seq
 from note_seq.notebook_utils import colab_play as play
 import numpy as np
 import tensorflow.compat.v1 as tf
 tf.disable_v2_behavior()

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit



Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
def upload():
   map = files.upload()
   list = []
   for key, val in map.iteritems():
     filename = os.path.join('/content/gansynth/midi', key)
     with open(filename, 'w') as file:
       file.write(val) 
       print('Writing the file {}'.format(filename))
       list.append(filename)
   return list

In [5]:
CHECKPOINT_DIR = 'gs://magentadata/models/gansynth/acoustic_only'
OP_DIR = '/content/gansynth/samples'
BATCH_SIZE = 16
SR = 16000

In [6]:
OP_DIR = util.expand_path(OP_DIR)
if not tf.gfile.Exists(OP_DIR):
  tf.gfile.MakeDirs(OP_DIR) 

In [7]:
tf.reset_default_graph() 
myflags = lib_flags.Flags({
     'batchSizeSchedule': [BATCH_SIZE],
     'tfds_data_dir': "gs://tfds-data/datasets",
})
model = lib_model.Model.load_from_path(CHECKPOINT_DIR, myflags)

	adam_beta1: 0.0
	adam_beta2: 0.99
	audio_length: 64000
	batchSizeSchedule: [16]
	batch_size_schedule: [8]
	d_fn: specgram
	data_normalizer: specgrams_prespecified_normalizer
	data_type: mel
	dataset_name: nsynth_tfds
	debug_hook: False
	discriminator_ac_loss_weight: 10.0
	discriminator_learning_rate: 0.0008
	fake_batch_size: 61
	fmap_base: 4096
	fmap_decay: 1.0
	fmap_max: 256
	g_fn: specgram
	gan:
	gen_gl_consistency_loss_weight: 0.0
	generator_ac_loss_weight: 10.0
	generator_learning_rate: 0.0008
	gradient_penalty_target: 1.0
	gradient_penalty_weight: 10.0
	kernel_size: 3
	latent_vector_size: 256
	mag_normalizer_a: 0.0661371661726
	mag_normalizer_b: 0.113718730221
	master: 
	normalizer_margin: 0.8
	normalizer_num_examples: 1000
	num_resolutions: 7
	p_normalizer_a: 0.8
	p_normalizer_b: 0.0
	ps_tasks: 0
	real_score_penalty_weight: 0.001
	sample_rate: 16000
	save_summaries_num_images: 10000
	scale_base: 2
	scale_mode: ALL
	simple_arch: False
	stable_stage_num_images: 800000
	start_heigh

Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.random.categorical` instead.


...

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Restoring parameters from gs://magentadata/models/gansynth/acoustic_only/stage_00012/model.ckpt-11000000


INFO:tensorflow:Restoring parameters from gs://magentadata/models/gansynth/acoustic_only/stage_00012/model.ckpt-11000000


In [8]:
 def midiLoad(path, minimumPitch=36, maximumPitch=84):
   midiPath = util.expand_path(path)
   noteSequence = note_seq.midi_file_to_sequence_proto(midiPath)
   pitches = np.array([n.pitch for n in noteSequence.notes])
   velo = np.array([nt.velocity for nt in noteSequence.notes])
   startTimes = np.array([nt.start_time for nt in 
   noteSequence.notes])
   endTimes = np.array([nt.end_time for nt in noteSequence.notes])
   valid = np.logical_and(pitches >= minimumPitch, pitches <= 
   maximumPitch)
   notes = {'pitches': pitches[valid],
            'velocities': velo[valid],
            'startTimes': startTimes[valid],
            'endTimes': endTimes[valid]}
   return noteSequence, notes

In [9]:
def createEnvelope(note_length, attack=0.010, release=0.3, sr=16000): 
   note_len = min(note_length, 3.0)
   attack = int(sr * attack)
   sustain = int(sr * note_len)
   release = int(sr * release)
   total = sustain + release
   env = np.ones(total)
   env[:attack] = np.linspace(0.0, 1.0, attack)
   env[sustain:total] = np.linspace(1.0, 0.0, release)
   return env 

In [10]:
def spectrogram(audioClip):
   min = np.min(36)
   max = np.max(84)
   minF = librosa.midi_to_hz(min)
   maxF = 2 * librosa.midi_to_hz(max)
   octaves = int(np.ceil(np.log2(maxF) - np.log2(minF)))    
   binsPerOctave = 36
   nBins = int(binsPerOctave * octaves)
   C = librosa.cqt(audioClip, sr=SR, hop_length=2048, fmin=minF,     
   n_bins=nBins, bins_per_octave=binsPerOctave)
   power = 10 * np.log10(np.abs(C)**2 + 1e-6)
   plt.matshow(power[::-1, 2:-2], aspect='auto', cmap=plt.cm.magma)
   plt.yticks([])
   plt.xticks([])

In [11]:
midi_file = "Arpeggio (Default)"

In [12]:
 midi_path = RIFF
 if midi_file == "Upload your own":
   try:
     fileList = upload()
     midi_path = fileList[0]
     noteSeqence, notes = midiLoad(midi_path)
   except Exception as e:
     print('Upload Cancelled')
 else:
   noteSequence, notes = midiLoad(midi_path)
   notes['startTimes'] *= 1.3
   notes['endTimes'] *= 1.3
 note_seq.plot_sequence(noteSequence) 

In [13]:
number_of_random_instruments = 10

In [14]:
pitchPreview = 60
num = number_of_random_instruments
pitches = [pitchPreview] * num
latent_vector = model.generate_z(num)
audio_notes = model.generate_samples_from_z(latent_vector, pitches)
for i, audio_note in enumerate(audio_notes):
   print("Instrument: {}".format(i))
   play(audio_note, sample_rate=16000)

generate_samples: generated 10 samples in 12.077441215515137s
Instrument: 0


Instrument: 1


Instrument: 2


Instrument: 3


Instrument: 4


Instrument: 5


Instrument: 6


Instrument: 7


Instrument: 8


Instrument: 9


In [ ]:
instruments = [0, 2, 4, 0]
times = [0, 0.3, 0.6, 1.0]
times[0] = -0.001
times[-1] = 1.0
z_instruments = np.array([latent_vector[i] for i in instruments])
t_instruments = np.array([notes['endTimes'][-1] * t for t in times])
z_notes = gu.get_z_notes(notes['startTimes'], z_instruments, t_instruments)
print('Generating {} samples...'.format(len(z_notes)))
audio_notes = model.generate_samples_from_z(z_notes, notes['pitches'])

Generating 128 samples...


In [ ]:
def combine_notes(audio, start, end, velo, sr=16000):
   numberOfNotes = len(audio)
   clipLen = end.max() + 3.0
   clip = np.zeros(int(clipLen) * sr)
   for t_start, t_end, velocity, i in zip(start, end, velo, range(numberOfNotes)):
     noteLen = t_end - t_start
     env = createEnvelope(noteLen) 
     length = len(env)
     audio_note = audio[i, :length] * env
     audio_note /= audio_note.max()
     audio_note *= (velocity / 127.0)
     clipStart = int(t_start * sr)
     clipEnd = int(clipStart + length)
     clip[clipStart:clipEnd] += audio_note
     clip /= audio.max()
     clip /= 1.25
   return clip

In [ ]:
ac = combine_notes(audio_notes,
                   notes['startTimes'],
                   notes['endTimes'],
                   notes['velocities'])

play(ac, sample_rate=SR) 
print('CQT Spectrogram:')
spectrogram(ac)